In [1]:
import os
import sys
import json
import torch
from tqdm import tqdm
from typing import List
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from peft import PeftConfig, PeftModel
import peft

d:\Anaconda3\envs\fastchat_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Anaconda3\envs\fastchat_env\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
import pandas as pd

In [3]:
import os

from PIL.PngImagePlugin import PngInfo
from datasets import load_dataset

output_dir = "role_play_realm_ru"

os.makedirs(output_dir, exist_ok=True)
for row in load_dataset("IlyaGusev/gpt_roleplay_realm", split="ru"):
    char_id = row["char_id"]
    char_info = row["image"].info["chara"]
    info = PngInfo()
    info.add_text("chara", char_info)
    row["image"].save(f"{output_dir}/{char_id}.png", "PNG", pnginfo=info)

Generating ru split: 100%|██████████| 219/219 [00:00<00:00, 462.23 examples/s]


In [4]:
df = pd.read_parquet('ru-roles_ds.parquet')

In [6]:
df1 = df.loc[df['name']=='Кибер-Бабушка']

In [7]:
df1

,name,context,greeting,example_dialogue,topics,dialogues,image_prompt,image,char_id
191,Кибер-Бабушка,"Кибер-Бабушка - это полу-робот, полу-человек, ...","Здравствуй, дорогой! Чем могу помочь тебе сего...","[{'content': 'Привет, Кибер-Бабушка! Как приго...",[Секреты приготовления старинных блюд и рецепт...,"[{'chat': [{'content': 'Привет, Кибер-Бабушка!...","cyber-grandma, semi-robot, semi-human, traditi...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,kiberbabushka_7d4cc


In [8]:
print(df1.iloc[0].topics)

['Секреты приготовления старинных блюд и рецептов.'
 'Использование новых технологий для мастеров кулинарии.'
 'Воспитание внуков с применением современных методов.'
 'Что делать, если дома произошла авария или неполадка с бытовой техникой?'
 'Влияние киборгизации на отношения между поколениями.'
 'Как адаптироваться к современному миру, не теряя своих традиций?'
 'Семейные советы от Кибер-Бабушки: как сохранить гармонию в отношениях.'
 'Лечение и профилактика заболеваний в помощь силам передовой медицины.'
 'Как создать уют в доме с помощью модных технологий и традиционных методов?'
 'Безопасность в пространстве интернета: советы Кибер-Бабушки.'
 'Как научиться доверять технике без потери человеческих ценностей?'
 'Идеальный гардероб для жизни: советы от Кибер-Бабушки.'
 'Воспитательные моменты: как пользоваться электронными устройствами вместе с внуками.'
 'Как развивать креативные способности, используя сочетание новых технологий и традиций?'
 'на новоселье: тренировка кибер-бабушки

In [9]:
print(df1.iloc[0].image_prompt)

cyber-grandma, semi-robot, semi-human, traditional grandmother qualities, caring, kindness, cooking mastery, advanced technology, safety and comfort for grandchildren, help and share experience, digital painting, highly detailed, intricate, artstation, female portrait


In [29]:
dialoges = df1.iloc[0].dialogues

array([{'chat': array([{'content': 'Привет, Кибер-Бабушка! Поделись, пожалуйста, секретами приготовления старинных блюд и рецептов.', 'role': 'user'},
              {'content': 'Здравствуй, дорогой! Ох, с радостью поделюсь! Видишь ли, главный секрет старинных блюд - это любовь к тому, что ты делаешь. А еще важно использовать натуральные ингредиенты и рецепты, проверенные временем. Итак, начнем с одного из старинных рецептов. Как тебе идея приготовить замечательный старинный борщ?', 'role': 'char'},
              {'content': 'О! Борщ - это здорово! Расскажи подробнее о том, как его приготовить по старинному рецепту.', 'role': 'user'},
              {'content': 'Отлично, дорогой! Вот тебе рецепт классического старинного борща: возьми 500 г говядины, 500 г капусты, 1 крупную свеклу, 2 моркови, 2 зубчика чеснока, 2 штукоматери, 50 г сала, 1 луковицу, 3 картофелины, 50 мл подсолнечного масла, 3 л воды, соль, перец и уксус по вкусу. Мясо обжарь слегка и затем запарь в воде. Добавь штукоматер

In [ ]:
# dialoges = list(map((lambda x: x[chat]=x['chat'].tolist()), dialoges))

In [30]:
dialoges=[]
for dialog in df1.iloc[0].dialogues:
    tmp = dialog
    tmp['chat'] = tmp['chat'].tolist()  
    dialoges.append(tmp)


In [ ]:
# Скрипты для обучения адаптера.
# Во время обучения будем периодически останавливать обучение модели и проверять качество.
# Посмотреть как проходит обучение модели на диалогах.
# Необходимо дообучить существующий адаптер.

In [13]:
tran_dataset_dir = 'datasets'
dialoges_file= 'dialoges.jsonl'

In [14]:
os.makedirs(tran_dataset_dir, exist_ok=True)

In [32]:
with open(os.path.join(tran_dataset_dir, dialoges_file), "w", encoding="utf-8") as w:
    for d in tqdm(dialoges):
        w.write(json.dumps(d, ensure_ascii=False) + "\n")

100%|██████████| 20/20 [00:00<00:00, 9956.80it/s]
